In [11]:
import pandas as pd
import pymysql
import datetime

2016Census_G02_SA_POA


In [ ]:
##Dont Use. Just for test purposes
import json
with open('C:\\Users\Arjun\Desktop\poa\Output_Table_names.txt','r') as f:
    myDictLoaded = json.load(f)
print(myDictLoaded[1][0])
tablenames = [i[0] for i in myDictLoaded]

In [13]:
# Creating a connection to the database.
def openMySqlConnection(host, port, dbname, user, password):
    conn = pymysql.connect(host, user=user,port=port,passwd=password, db=dbname)
    return conn


In [14]:
# Closes the connection to the database.
def closeMySqlConnection(conn):
    conn.close()

In [15]:
# Creating a new user in the Tableau Database and granting them basic privileges to view present tables.
def createUser(userName, userPass, tablenames):
    try:
        conn = newConnection()
        stmntNewUser = "CREATE USER IF NOT EXISTS '" + userName + "'@'%'" + " IDENTIFIED BY '" + userPass + "';"
        with conn.cursor() as cursor:
            sql = stmntNewUser
            cursor.execute(sql)
            
        for i in range(len(tablenames)):
            with conn.cursor() as cursor:
                stmntNewUser = "Grant SELECT on DataSparkDataBase."+tablenames[i]+" TO '"+userName+"'@'%' ;"
                cursor.execute(stmntNewUser)
        
    finally:
        closeMySqlConnection(conn)


In [35]:
# Creating a user specified view
def createView(tablenames, columnNames, viewName, userName, databaseName):
    now = datetime.datetime.now()
    #Adding time of view creation to the view name to reduce the probability that 2 users have the same view name
    a = "_"+str(now.day)+"_"+str(now.month)+"_"+str(now.year)+"_"+str(now.hour)+"_"+str(now.minute)+"_"+str(now.second)
    
    try:
        conn = newConnection()
        conn1 = newConnection()
        #The Following Creates  the view statement user has requested
        stmt = "Create view " + viewName + a + " as"
        selectstmt = " select " + tablenames[0]+".POA_CODE_2016, "
        
        for i in columnNames:
            if(i!="POA_CODE_2016"):
                selectstmt += i + ", "
        selectstmt=selectstmt[:-2]
        selectstmt+= " from "
        
        for i in tablenames:
            selectstmt+= i + ", "
        selectstmt=selectstmt[:-2]
        if(len(tablenames)>1):
            selectstmt+= " where "
            for j in range(len(tablenames)-1):
                selectstmt+= tablenames[0]+".POA_CODE_2016 = " + tablenames[j+1] + ".POA_CODE_2016 AND "
        
        selectstmt=selectstmt[:-4]
        stmt= stmt + selectstmt
        pvtstmt = ''
        #Creating the pivoted view using union all command as MYSQL doesn't have the Pivot funciton inbuilt
        
        for i in range(len(columnNames)-1):
            pvtstmt = pvtstmt+ 'SELECT POA_CODE_2016, \''+columnNames[i+1]+'\' Attributes, '+columnNames[i+1]+' val from ('+selectstmt+') p union all '
        pvtstmt=pvtstmt[:-11]
        pvtstmt = "Create view pivoted_"+ viewName + a + " as " + pvtstmt
        
        with conn.cursor() as cursor:
            cursor.execute(stmt)
            cursor.execute(pvtstmt)
        # Granting permissions to both the views created
        grantPermissions(databaseName, viewName+a, userName)
        grantPermissions(databaseName, 'pivoted_'+viewName+a, userName)
    finally:
        closeMySqlConnection(conn)
        return stmt


In [17]:
#Gives permissions in a specif view
def grantPermissions(databaseName, viewName, userName):
    stmnt = "GRANT ALL ON " + databaseName + "." + viewName + " TO '" + userName + "'@'%';"
    try:
        conn = newConnection()
        with conn.cursor() as cursor:
            sql = stmnt
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)


In [18]:
#This function revokes permissions of a user
def revokePermissions(userName):
    stmnt = "REVOKE ALL PRIVILEGES ON *.* FROM '" + userName + "'@'%';"
    try:
        conn = newConnection()
        with conn.cursor() as cursor:
            sql = stmnt
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)

In [19]:
#This creates a connection to the database
def newConnection():
    host = "datasparkdb.cbbvbxlm9nap.us-east-1.rds.amazonaws.com"
    port = 3306
    dbname = "DataSparkDataBase"
    user = "aarasu"
    password = "aabbccdd1234"
    conn = openMySqlConnection(host, port, dbname, user, password)
    return conn

In [20]:
#This function will 
def getTableNames(tclist):
    tableList = []
    uTableList = [item[0] for item in tclist]
    for i in uTableList:
        if i not in tableList:
            tableList.append(i)
    return tableList
            

In [21]:
#This returns a list of all columns of the tables present in the list
def getColumnNames(tclist):
    uColumnList = [item[1] for item in tclist]
    return uColumnList

In [22]:
#Retrieves the Views accessable by the user
def getViewNames(username, password):
    try:
        conn = pymysql.connect("datasparkdb.cbbvbxlm9nap.us-east-1.rds.amazonaws.com", user=username,port=3306,passwd=password, db="DataSparkDataBase")
        stmnt = "SHOW FULL TABLES IN DataSparkDataBase WHERE TABLE_TYPE LIKE 'VIEW';"
        with conn.cursor() as cursor:
            sql = stmnt
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)
    viewNames = [i[0] for i in cursor.fetchall()]
    return viewNames
    
    
    

In [23]:
#Test
a=getViewNames("aarasu", "aabbccdd1234")
print(a)

The connection is closed
['G04V', 'G05V', 'ThursdayTest_19_4_2018_13_42_35_', 'View1_19_4_2018_15_13_25_', 'erick_selected_tables_columns', 'german2_selected_tables_columns', 'german_selected_tables_columns', 'm10_selected_tables_columns', 'mustafa2_selected_tables_columns', 'pivoted_v1', 'pivoted_v33_24_4_2018_18_40_45', 'pivoted_v34_24_4_2018_18_42_41', 'testztuesday_24_4_2018_17_24_26_', 'v33_24_4_2018_18_40_45', 'v34_24_4_2018_18_42_41', 'viewft1', 'viewgen3', 'viewgen5_18_4_2018_23_25_24_', 'viewgen5_18_4_2018_23_26_15_']


In [24]:
#Test
print(a[:][0])

G04V


In [25]:
# This funciton deletes the views of a user.
# viewname is a list of views
def viewsDelete(username, password, viewname): 
    try:
        conn = pymysql.connect("datasparkdb.cbbvbxlm9nap.us-east-1.rds.amazonaws.com", user=username,port=3306,passwd=password, db="DataSparkDataBase")
        stmnt = "Drop VIEW if exists "
        for i in viewname:
            stmnt= stmnt + i +", " 
        stmnt = stmnt[:-2]
        with conn.cursor() as cursor:
            sql = stmnt
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)
    return "views Deleted"
     

In [26]:
#This function deletes all the views created by the user and deletes the user itself.
def userDelete(username, password):
    try:
        conn = newConnection()
        stmnt_1 = "DROP USER IF EXISTS '"+username+"'@'%' ;"
        userViews = getViewNames(username, password)
        UserViews_deleteStatus = viewsDelete(username, password, userviews)
        
        with conn.cursor() as cursor:
            sql = stmnt_1
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)
    
            
            
            
            

In [27]:
# This function updates the user's password.
def userUpdatePassword(username, newPassword):
    try:
        conn = newConnection()
        stmnt = "ALTER USER '"+username+"'@'%' identified by '"+newPassword+"';"
        
        with conn.cursor() as cursor:
            sql = stmnt
            cursor.execute(sql)
    finally:
        closeMySqlConnection(conn)
    a = username+ " password updated"
    return a
            

In [28]:
u = userUpdatePassword('dummyuser1', 'abcdef1234')
print(u)

The connection is open
The connection is closed
dummyuser1 password updated


In [30]:
userDelete("dummyuser1","abcdef1234")

The connection is open
The connection is closed
The connection is closed


In [31]:
# Checking whether a username exists in the list of active users.
def checkUser(username):
    try:
        conn = newConnection()
        stmntNewUser = "SELECT EXISTS(SELECT 1 FROM mysql.user WHERE user = '"+username+"')"

        with conn.cursor() as cursor:
            sql = stmntNewUser
            #print(sql)
            cursor.execute(sql)
            viewNames = [i[0] for i in cursor.fetchall()]

    finally:
        closeMySqlConnection(conn)
        return viewNames[0]

In [32]:
#Test
createUser('dummyUser1', 'aabbccdd1234', tablenames)

The connection is open
CREATE USER IF NOT EXISTS 'dummyUser1'@'%' IDENTIFIED BY 'aabbccdd1234';
The connection is closed
User created


In [36]:
#Test
a = createView(['2016Census_G02_SA_POA','2016Census_G04B_SA_POA'],['Median_rent_weekly', 'Median_tot_fam_inc_weekly', 'Age_yr_61_M', 'Age_yr_85_89_M', 'Age_yr_95_99_P'], 'VVii', 'dummyUser1', 'DataSparkDataBase' )


The connection is open
The connection is open
hi1
Create view VVii_25_4_2018_17_7_28 as select 2016Census_G02_SA_POA.POA_CODE_2016, Median_rent_weekly, Median_tot_fam_inc_weekly, Age_yr_61_M, Age_yr_85_89_M, Age_yr_95_99_P from 2016Census_G02_SA_POA, 2016Census_G04B_SA_POA where 2016Census_G02_SA_POA.POA_CODE_2016 = 2016Census_G04B_SA_POA.POA_CODE_2016 
Create view pivoted_VVii_25_4_2018_17_7_28 as SELECT POA_CODE_2016, 'Median_tot_fam_inc_weekly' Attributes, Median_tot_fam_inc_weekly val from ( select 2016Census_G02_SA_POA.POA_CODE_2016, Median_rent_weekly, Median_tot_fam_inc_weekly, Age_yr_61_M, Age_yr_85_89_M, Age_yr_95_99_P from 2016Census_G02_SA_POA, 2016Census_G04B_SA_POA where 2016Census_G02_SA_POA.POA_CODE_2016 = 2016Census_G04B_SA_POA.POA_CODE_2016 ) p union all SELECT POA_CODE_2016, 'Age_yr_61_M' Attributes, Age_yr_61_M val from ( select 2016Census_G02_SA_POA.POA_CODE_2016, Median_rent_weekly, Median_tot_fam_inc_weekly, Age_yr_61_M, Age_yr_85_89_M, Age_yr_95_99_P from 2016Cen

In [38]:
#Test
a=getViewNames("dummyUser1", "aabbccdd1234")
print(a)

The connection is closed
['VVii_25_4_2018_17_7_28', 'pivoted_VVii_25_4_2018_17_7_28', 'vii_25_4_2018_17_5_14']
